<a href="https://colab.research.google.com/github/amun83/Koscom-Financial-Data-Bootcamp/blob/master/FD104_Text_Mining_OilPrice_com_News_Articles_Step_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. Scraping News Articles: https://oilprice.com/

In [0]:
ON_COLAB  = True
DATA_PATH = ''

In [0]:
!pip install -q --upgrade pandas_datareader
!pip install -q --upgrade finance-datareader
!pip install -q --upgrade html5lib
!pip install -q --upgrade lxml
!pip install -q --upgrade seaborn
!pip install -q --upgrade gspread

In [0]:
# utilities
import datetime
import re
import csv
from IPython.display import display

# web scraping
import requests
import urllib.parse
from bs4 import BeautifulSoup as bs
import pandas_datareader as pdr
import FinanceDataReader as fdr

# Math/Stat
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
#import math

# Machine Learning
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM

# visualization
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
if ON_COLAB :
  from google.colab import drive
  drive.mount('/gdrive')
  DATA_PATH = '/gdrive/My Drive/sandbox/'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
pattern = re.compile(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s(\d\d),\s(\d{4})')
# Example: By Julianne Geiger - Dec 14, 2018, 8:00 PM CST

In [0]:
def get_news_urls_by_page(page_num):
  try:
    get_news_urls_by_page.counter += 1
  except:
    get_news_urls_by_page.counter = 0
    
  page_url = 'https://oilprice.com/Latest-Energy-News/World-News/Page-' + str(page_num) + '.html'
  # https://oilprice.com/Latest-Energy-News/World-News/Page-865.html

  try:
    r = requests.get(page_url)
    if get_news_urls_by_page.counter % 10 == 0:
      print('Request(GET) >> OK: ' + page_url)
  except:
    print('Request(GET) >> Failed: ' + page_url)
    return None

  soup = bs(r.text)
  items = soup.find_all('div', {'class':'categoryArticle__content'})

  if not items:
    print('Request(GET) >> Empty Page: ' + page_url)
    return None
  
  return [item.find('a', href = True)['href'] for item in items]

In [0]:
def get_news_content(url):
  try:
    get_news_content.counter += 1
  except:
    get_news_content.counter = 0
  
  try:
    r = requests.get(url)
    if get_news_content.counter % 100 == 0:
      print('[' + str(get_news_content.counter) + '] Request(GET) >> OK: ' + url)
  except:
    print('[' + str(get_news_content.counter) + '] Request(GET) >> Failed: ' + url)
    return None

  soup = bs(r.text)

  title = soup.find('h1').text

  date = soup.find('span', {'class':'article_byline'}).text

  match = re.search(pattern, date)
  mmm, dd, yyyy = match.groups(0)
  date = yyyy + "-" + mmm + "-" + dd

  raw_content = soup.find('div', {'id':'news-content'}).find_all('p')
  content = ' '.join([p.text.strip() for p in raw_content])
  
  return date, title, content

In [0]:
news_urls =[]

for i in range(1, 1000):
  urls = get_news_urls_by_page(i)
  if not urls:
    break
    
  news_urls += urls

Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-1.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-11.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-21.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-31.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-41.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-51.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-61.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-71.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-81.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-91.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-101.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-

In [0]:
print(len(news_urls))
print(news_urls[-1])

8652
https://oilprice.com/Latest-Energy-News/World-News/Romania-Posts-2011-2035-National-Energy-Strategy-Online.html


In [0]:
with open(DATA_PATH + 'oilprice.com_news_urls.csv', 'w') as f:
  fw = csv.writer(f)
  fw.writerows([[url] for url in news_urls])

In [0]:
news = [get_news_content(url) + (url.split('/')[-1],) for url in news_urls]

[0] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/France-Extends-State-Guarantee-For-Total-Participated-Russian-LNG-Projects.html
[100] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Pemex-Doubles-Reserve-Estimate-At-One-Of-Worlds-Largest-Finds.html
[200] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/South-Korea-To-Restart-Iran-Oil-Imports-At-4M-Barrels-Per-Month.html
[300] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Kinder-Morgan-Cancels-Gas-Pipeline-Project.html
[400] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Total-Bets-On-Deepwater-Oil-Gas-To-Boost-Production.html
[500] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Major-Japanese-Refiners-Suspend-Iranian-Oil-Imports-For-October.html
[600] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Indias-Reliance-Declares-Force-Majeure-At-Major-Export-Refinery.html
[700

In [0]:
print(len(news))
display(news[-1])

8652


('2011-Jun-15',
 'Romania Posts 2011-2035 National Energy Strategy Online',
 "Romania’s Ministry of Economy, Trade and Business Environment (MECMA) has posted its 2011-2035 energy strategy online for discussion. The document is intended to open discussion about Romania’s energy policies for 2011-2035 by providing international context, Romania’s current energy policies, its main targets and trends as well as forecasts concerning the country’s energy sector and measures for meeting the priority targets, Romania’s Agerpres reported. MECMA comments that updating Romania’s energy strategy, as required in the government’s 2009-2012 program, the first target of chapter 17’s “Energy and Mineral Resources” requires a deep analysis of the country’s current energy sector, as well developing potential solutions. MECMA noted, “Both the economic crisis in the past years and the new law packages adopted in the European Union in the field of energy and climate changes have deeply modified the context

In [0]:
with open(DATA_PATH + 'oilprice.com_news.csv', 'w') as f:
  fw = csv.writer(f)
  fw.writerows(news)